In [1]:
import numpy as np
import cv2

In [2]:
def show_image(img, window_name='image'):
    cv2.namedWindow(window_name)
    cv2.startWindowThread()
    cv2.imshow(window_name, img)
    cv2.waitKey(10000)
    cv2.destroyAllWindows()

In [3]:
def get_SE(img, SE_size_factor=0.02):
    nrows, ncols = img.shape[0], img.shape[1]
    ROI_area = nrows*ncols
    SE_size = SE_size_factor*np.sqrt(ROI_area/np.pi)
    SE = np.ones((SE_size,SE_size))
    lam = 5*SE_size
    return SE, lam

In [4]:
def binarize(img, threshold=127, vizualize=True):
    ret, binarized = cv2.threshold(img, threshold, 255, cv2.THRESH_BINARY)
    binarized = binarized[:,:,0]
    if(vizualize):
        show_image(binarized)
    return binarized

In [5]:
def fill_image(img, lam=20, vizualize=True):
    filled = img.copy()
    filled_small = np.zeros(example.shape, dtype='uint8')
    img2, contours, hierarchy = cv2.findContours(filled,cv2.RETR_CCOMP,cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        #Fill the original image for all the contours
        cv2.drawContours(filled, [cnt], 0, 255, -1)
        #If it's a small contour, draw it to the noise-image
        if cv2.contourArea(cnt) < lam:
            cv2.drawContours(filled_small, [cnt], 0, 255, -1)
            
    if(vizualize):
        show_image(filled, 'filled image')
        show_image(filled_small, 'filled small elements')
    
    return filled, filled_small

In [6]:
def get_holes(img, lam=20, vizualize=True):
    if(vizualize):
        show_image(img, 'original')

    #retrieve the filled image and the filled small elements
    filled, filled_small = fill_image(img, lam, vizualize)
    #get all the holes (including those that are noise)
    all_the_holes = cv2.bitwise_and(filled, cv2.bitwise_not(img))
    #Substract the noise elements
    theholes = cv2.bitwise_and(all_the_holes, cv2.bitwise_not(filled_small))

    if(vizualize):
        show_image(all_the_holes, 'holes with noise')
        show_image(theholes, 'holes without noise')
    
    return filled, theholes

In [7]:
def get_islands(img, lam=20, vizualize=True):
    invimg = cv2.bitwise_not(img)
    invfilled, islands = get_holes(invimg, lam, vizualize)
    return invfilled, islands

In [ ]:
def get_protrusions(img, filled, SE, lam):
    filled, filled_small = fill_image(img, lam)
    fill_big = cv2.bitwise_and(filled, cv2.bitwise_not(cv2.bitwise_and(img, filled_small)))

In [8]:
#Load the image
path_to_image = '../../../TestData/Binary/Binary_holes.png'
holes = binarize(cv2.imread(path_to_image))
path_to_image_islands = '../../../TestData/Binary/Binary_islands.png'
islands = binarize(cv2.imread(path_to_image_islands))
path_to_image_example = '../../../TestData/Binary/Binary_all_types_noise.png'
example = binarize(cv2.imread(path_to_image_example))
path_to_image_indpred = '../../../TestData/Binary/Binary_indentations_protrusions.png'
indpred = binarize(cv2.imread(path_to_image_indpred))
path_to_image_nested = '../../../TestData/Binary/Binary_nested.png'
nested = binarize(cv2.imread(path_to_image_nested))

In [9]:
SE, lam = get_SE(example, 0.05)
print 'lambda is: %f' % lam
#holesim_filled, holesim_holes = get_holes(holes)
#islandsim_filled, islandsim_holes = get_holes(islands)
alltypes_filled, alltypes_holes = get_holes(example, lam)
#nested_filled, nested_holes = get_holes(nested)

lambda is: 36.108133


In [40]:
holesim_filledinv, holesim_islands = get_islands(holes)
islandsim_filledinv, islandsim_islands = get_islands(islands)
alltypes_filledinv, alltypes_islands = get_islands(example)

In [12]:
#Find the connected components (similar to bwlabel)
retval, labels = cv2.connectedComponents(binarized[:,:,0])

In [26]:
#An ugly transformation to see the CC's in different colors
dim = np.ceil(np.power(retval, 1.0/3))
step = int(np.floor(255/dim))
print dim, step
rvalues = np.arange(int(np.floor(step/2)),256, step)
colormapping = dict([(i,[rvalues[np.mod(i, dim)], 
                         rvalues[np.floor(np.mod(i, dim*dim)/dim)], 
                         rvalues[np.floor(i/dim)]]) for i in xrange(1, retval)])
colormapping[0] = [0,0,0]
colored_labels = np.array([[colormapping[i] for i in x]  for x in labels], dtype='uint8')
show_image(colored_labels)

2.0 127


/home/dafne/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:8: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [15]:
#Experiments for indentation removal
#TODO: there are still some boundaries shown in fill_big!
SE, lam = get_SE(example, 0.15)
#print SE
img = example
filled, filled_small = fill_image(img, lam)
fill_big = cv2.bitwise_and(filled, cv2.bitwise_not(cv2.bitwise_and(img, filled_small)))
show_image(fill_big)
#tophat = cv2.morphologyEx(fill_big, cv2.MORPH_TOPHAT, SE)